In [10]:
import pandas as pd
import glob
import os
import keras

In [2]:
#save the models to model_folder
model_folder=''
#output folder of trajactory got from the simulation
output_folder=''
#create energy_folder to save the prediction results
energy_folder=''

# Select scenario for weather condition

In [3]:
#weather option: rainy, sunny, windy, and snowy
weather = 'sunny'

# Load microscopic energy eatimation model for different vehicle classes


In [4]:
model_diesel =  keras.models.load_model(f'{model_folder}/diesel.model')
model_hybrid =  keras.models.load_model(f'{model_folder}/hybrid.model')
model_electric =  keras.models.load_model(f'{model_folder}/electric.model')

# Predict energy consumption of each trip 

In [7]:
path = f'{output_folder}'                    
all_files = glob.glob(os.path.join(path, "Trajectory*.csv")) 
colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass']
for f in all_files:
    trip=pd.read_csv(f,names=colnames, header=None)
    #trip=pd.read_csv(f)
    trip = trip[trip.speed != 'speed']
    trip = trip[trip.acceleration != 'acceleration']
    trip['speed']=trip['speed'].astype(float)*0.036 #km/h
    #find the vehicle class of this trip based on the column "actorConfig_ref"
    vtype = ['Gillig_LF_2014','Gillig_HF_2006','Gillig_HF_2002','Gillig_HF_1998','Gillig_Hybrids_2014',
             'Gillig_Hybrids_2009','Gillig_LF_2009','BYD_BEV']
    diesel = ['Gillig_LF_2014','Gillig_HF_2006','Gillig_HF_2002','Gillig_HF_1998','Gillig_LF_2009']
    hybrid = ['Gillig_Hybrids_2014', 'Gillig_Hybrids_2009']
    electric = ['BYD_BEV']
    if trip['actorConfig_ref'].iloc[2] in diesel:
        Vclass = 'diesel'
    elif trip['actorConfig_ref'].iloc[2] in hybrid:
        Vclass = 'hybrid'
    elif trip['actorConfig_ref'].iloc[2] in electric:
        Vclass = 'electric'
    if Vclass == 'electric':
        trip['acceleration']=trip['acceleration'].astype(float)*0.001*3.28084
        #ft/s2
    else:
        trip['acceleration']=trip['acceleration'].astype(float)*0.001 
        #m/s2
    if weather == 'rainy':
        trip['temperature']= 60 
        trip['humidity']= 0.9 #range[0,1]
    if weather == 'sunny':
        trip['temperature']= 70
        trip['humidity']= 0.5
    if weather == 'windy':
        trip['temperature']= 60
        trip['humidity']= 0.6
    if weather == 'snowy':
        trip['temperature']= 30
        trip['humidity']= 0.4
    trip['roadGrade']= 0
    input2esti=trip[['speed','acceleration','temperature','humidity','roadGrade']]
    #prdiction 
    if Vclass == 'electric':
        pre = model_electric.predict(input2esti) 
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1)
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/33.7  #gal
        with open(f'{energy_folder}' + Vclass +'_'+ f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)    
    elif Vclass == 'hybrid':
        pre = model_hybrid.predict(input2esti)
        pre[pre<0]=0
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1) 
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/3600  #gal
        with open(f'{energy_folder}' + Vclass +'_'+ f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    elif Vclass == 'diesel':
        pre = model_diesel.predict(input2esti)
        pre[pre<0]=0
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1)    
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/3600  #gal
        #write the predicted results 
        with open(f'{energy_folder}' + Vclass +'_'+ f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    

# Estimate energy consumption of each route

In [9]:
Route = ['Route1','Route3','Route4','Route8','Route9','Route10A','Route10G','Route16','Route21','Route33','Route34']
path = f'{energy_folder}'
route_summary = pd.DataFrame()
b=0
for r in Route:
    allfiles = glob.glob(os.path.join(path, "*"+r+"*.csv")) 
    if len(allfiles) > 0:
        colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass','temperature','humidity','roadGrade','energyrate','energy_gal']
        route_energy = pd.DataFrame()
        c=0
        for f in allfiles:
            df=pd.read_csv(f,names=colnames, header=None)
            df['speed']=pd.to_numeric(df['speed'],errors='coerce')
            df['energy_gal']=pd.to_numeric(df['energy_gal'],errors='coerce')    
            df['speed_mile_s']=df['speed']*0.000172603  #mile/s 
            energy=df['speed_mile_s'].sum()/df['energy_gal'].sum() #mi/gal
            sum_gal = df['energy_gal'].sum()
            route_energy.loc[c,'energy_mpg']=energy
            route_energy.loc[c,'sum_energy_gal']=sum_gal
#             with open(f'{energy_folder}'+'_'+ r + 'energy.csv', 'w', newline='') as wFile:
#                 route_energy.to_csv(wFile, index = False)
            c+=1
        route_energy['trip_id']=[f[-21:-8] for f in allfiles]
        route_summary.loc[b,'sum_energy_gal']=route_energy['sum_energy_gal'].sum()
        route_summary.loc[b,'route']=r
        b+=1
print(route_summary)

   sum_energy_gal     route
0       81.008639    Route1
1       25.053928    Route3
2       47.493702    Route4
3        5.756740    Route8
4       22.475999    Route9
5       12.403156  Route10A
6       17.205794  Route10G
7       14.239992   Route16
8        5.812320   Route21
9        6.766614   Route33
